In [1]:
!pip install pypdf
!pip install PyMuPDF
!pip install frontend
!pip install tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting 

In [2]:
!mkdir static

In [3]:
import pypdf
import urllib.request
import fitz
from PIL import Image

def gettext(pdf):
    text=""
    for i in pdf.pages:
        text+=i.extract_text()
    return text

def getims(pdf):
    li=[]
    for page in pdf.pages:
          for image in page.images:
              li.append(image.name)
              with open(image.name, "wb") as fp:
                  fp.write(image.data)
    return li

def first(filename,page):
    file_handle = fitz.open(filename)
    page = file_handle[page]
    page_img = page.get_pixmap()
    page_img.save('PDF_page.png')
    img = Image.open('PDF_page.png')
    return img

In [4]:
# pdflink='https://med.und.edu/education-training/radiology/_files/docs/xray-film-reading-made-easy.pdf'
# urllib.request.urlretrieve(pdflink, "a.pdf")
# filename="1.pdf"
# read = open(filename, 'rb')
# pdf = pypdf.PdfReader(read)

In [4]:
from transformers import pipeline

labels=["Legal Document", "Medical Report", "Financial Statements", "Educational Material", "Business Document","News Article","Technical Document", "Creative Writing Document","Scientific Research Paper", "Government Document"]
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [14]:
scores=[]
for i in range(1,11):
  filename=f"{i}.pdf"
  read = open(filename, 'rb')
  pdf = pypdf.PdfReader(read)

  sequence_to_classify = gettext(pdf)
  sequence_to_classify = sequence_to_classify[:2500]
  output=classifier(sequence_to_classify, labels)
  sc={}
  for i in range(10):
    sc[output["labels"][i]]=output["scores"][i]
  scores.append(sc)
  print("done")

done
done
done
done
done
done
done
done
done
done


In [15]:
import copy
scores3=copy.deepcopy(scores)

In [8]:
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [59]:
scores2=[]
for i in range(1,11):
  image = first(f"{i}.pdf",0)
  image2 = first(f"{i}.pdf",1)

  inputs = processor(text=["Legal Document", "Medical Report", "Financial Statements", "Educational Material", "Business Document","News Article","Technical Document", "Creative Writing Document","Scientific Research Paper", "Government Document"], images=image, return_tensors="pt", padding=True)
  inputs2= processor(text=["Legal Document", "Medical Report", "Financial Statements", "Educational Material", "Business Document","News Article","Technical Document", "Creative Writing Document","Scientific Research Paper", "Government Document"], images=image2, return_tensors="pt", padding=True)

  outputs = model(**inputs)
  outputs2 = model(**inputs2)
  logits_per_image = outputs.logits_per_image
  logits_per_image2 = outputs2.logits_per_image
  probs = logits_per_image.softmax(dim=1)
  probs2 = logits_per_image2.softmax(dim=1)
  a = probs.detach().numpy()
  b = probs2.detach().numpy()
  sc={}
  for i in range(10):
    sc[labels[i]]=a[0][i]/2+b[0][i]/2
  scores2.append(sc)

In [70]:
scores=copy.deepcopy(scores3)

In [71]:
for i in range(10):
  for j in list(scores[i].keys()):
    scores[i][j]=0.75*scores[i][j]+0.25*scores2[i][j]
    if scores2[i][j]>0.7:
      scores[i][j]=1
    if scores3[i][j]>0.7:
      scores[i][j]=1
  print(max(scores[i], key=scores[i].get))

Educational Material
Creative Writing Document
Legal Document
Legal Document
Technical Document
Government Document
Financial Statements
News Article
Medical Report
Scientific Research Paper


In [72]:
for i in range(10):
  print(max(scores2[i], key=scores2[i].get))

Scientific Research Paper
Creative Writing Document
Technical Document
Legal Document
Technical Document
Government Document
Financial Statements
Technical Document
Medical Report
Scientific Research Paper


In [73]:
for i in range(10):
  print(max(scores3[i], key=scores3[i].get))

Educational Material
Creative Writing Document
Legal Document
Technical Document
Technical Document
Legal Document
Financial Statements
News Article
Medical Report
Educational Material


Filename   Mapping

1.pdf  Education Document

2.pdf  Creative Writing Document

3.pdf  Legal Document

4.pdf  Business Document

5.pdf  Technical Document

6.pdf  Government Document

7.pdf  Financial Document

8.pdf  News Document

9.pdf  Medical Document

10.pdf  Scientific Research Papers Document